In [1]:
### PREPARE DF FOR DARK SKY API ######
import sqlite3
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()

import pandas as pd
import seaborn as sns

c.execute('''SELECT * FROM Matches WHERE season = 2011;''')
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
print(len(df))
df.head()

992


,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D


In [2]:
# Remove unwanted columns #
# df.drop(['Div','FTHG','FTAG','FTR','AwayTeam','HomeTeam','Season'],axis = 1, inplace = True)

In [3]:
df.head()

,Match_ID,Date
0,1092,2012-03-31
1,1093,2011-12-11
2,1094,2011-08-13
3,1095,2011-11-27
4,1096,2012-02-18


In [9]:
# df.set_index('Match_ID',inplace = True)

In [10]:
# weather_df = df

In [11]:
# weather_df.head()

,Date
Match_ID,
1092,1333152000
1093,1323561600
1094,1313193600
1095,1322352000
1096,1329523200


In [2]:
#change date to unix date time for use in api call
df['Date'] = pd.to_datetime(df['Date'])
df['Unix_Date'] = (df['Date'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

In [3]:
df.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Unix_Date
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,1333152000
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,1323561600
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,1313193600
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,1322352000
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,1329523200


In [4]:
#### WEATHER API #####
df.Unix_Date.head()

0    1333152000
1    1323561600
2    1313193600
3    1322352000
4    1329523200
Name: Unix_Date, dtype: int64

In [4]:
#imports for api call
from bs4 import BeautifulSoup
import requests
import json
import pprint
date_test = ['1310688000','1336262400','1336867200']

In [6]:
date_test

['1310688000', '1336262400', '1336867200']

In [5]:
from apikey import *

In [11]:
#https://api.darksky.net/forecast/[key]/[latitude],[longitude],[time]?exclude=# format

# url = 'https://api.darksky.net/forecast'
# key = API_Key
# latitude = '52.52437'
# longitude = '13.41053'
# date = dates
# exclude = ('minutely','hourly','currently','alerts','flags')


# good_url = "{}/{}/{},{},{}?exclude={}".format(url,key,latitude, longitude, date, exclude)
# good_url


NameError: name 'dates' is not defined

In [14]:
#prep for api call test
# url = 'https://api.darksky.net/forecast/7bae2b226c09c124a036c0499d4f4456/52.52437,13.41053,1333152000?exclude=minutely,hourly,daily,alerts,flags'
# API_Key = '7bae2b226c09c124a036c0499d4f4456'
# payload = {'key':API_Key,'latitude':'52.52437','longitude': '13.41053','time':df['Date'],'exclude': ['minutely','hourly','daily','alerts','flags']}

def api_call(dates):
    url = 'https://api.darksky.net/forecast'
    key = API_Key
    latitude = '52.52437'
    longitude = '13.41053'
    date = dates
    exclude = ('minutely','hourly','currently','alerts','flags')
    good_url = "{}/{}/{},{},{}?exclude={}".format(url,key,latitude, longitude, date, exclude)
    
    response = requests.get(good_url)
    print(response)
    weather = response.json()['daily']['data'][0]['icon']
    return weather



In [22]:
response = requests.get(good_url)
print(response)
print(response.json()['daily']['data'][0]['icon'])

NameError: name 'good_url' is not defined

In [99]:
#keys from the api call to see what we want
response.json().keys()

dict_keys(['latitude', 'longitude', 'timezone', 'currently', 'offset'])

In [98]:
#call the weather
response.json()['daily']['summary']

KeyError: 'daily'

In [ ]:
#get weather from list of dates per match id


In [49]:
api_call(date_test)

<Response [400]>


KeyError: 'daily'

In [7]:
#get the weather for all the dates

def get_all_weather(dates):
    weather_dict = {}
    for date in dates:
        weather_data = api_call(date)
        
        weather_dict.update({date: weather_data})
    return weather_dict
#         weather_dict[date] = weather_data
# #         weather_dict['date'] = response.json()['daily']['data'][0]['icon']
#     return weather_dict


In [19]:
def adding_weather_to_df(dates):
        weather_dict = get_all_weather(dates)
        wdf = pd.DataFrame(weather_dict, index = ['weather'])
        wdf2 = pd.DataFrame.transpose(wdf)
        wdf2 = wdf2.reset_index()
        wdf2.rename(columns={'index': 'Unix_Date'}, inplace = True)
        df = pd.merge(df, wdf2, how = 'left', on = 'Unix_Date')
        return df.head()

In [15]:
adding_weather_to_df(df.Unix_Date)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


TypeError: merge() got an unexpected keyword argument 'inplace'

In [18]:
weather_dict

NameError: name 'weather_dict' is not defined

In [10]:
# get_all_weather(df.Unix_Date)
weather_dict = get_all_weather(df.Unix_Date)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [49]:
import json
json.dumps(weather_dict, indent = 2)

'{\n  "1333152000": "rain",\n  "1323561600": "partly-cloudy-day",\n  "1313193600": "partly-cloudy-day",\n  "1322352000": "partly-cloudy-day",\n  "1329523200": "partly-cloudy-day",\n  "1327017600": "rain",\n  "1328313600": "partly-cloudy-day",\n  "1334966400": "partly-cloudy-day",\n  "1316304000": "rain",\n  "1319328000": "rain",\n  "1317427200": "clear-day",\n  "1330732800": "clear-day",\n  "1314403200": "rain",\n  "1331942400": "clear-day",\n  "1320537600": "rain",\n  "1336176000": "rain",\n  "1334102400": "partly-cloudy-day",\n  "1324080000": "rain",\n  "1328227200": "partly-cloudy-day",\n  "1319846400": "clear-day",\n  "1327190400": "rain",\n  "1322870400": "rain",\n  "1334361600": "clear-day",\n  "1332633600": "partly-cloudy-day",\n  "1331337600": "partly-cloudy-day",\n  "1333756800": "partly-cloudy-day",\n  "1321660800": "partly-cloudy-day",\n  "1318550400": "clear-day",\n  "1316822400": "clear-day",\n  "1335571200": "clear-day",\n  "1324166400": "rain",\n  "1330646400": "partly-c

In [12]:
#checking the dictionary
weather_dict

{1333152000: 'rain',
 1323561600: 'partly-cloudy-day',
 1313193600: 'partly-cloudy-day',
 1322352000: 'partly-cloudy-day',
 1329523200: 'partly-cloudy-day',
 1327017600: 'rain',
 1328313600: 'partly-cloudy-day',
 1334966400: 'partly-cloudy-day',
 1316304000: 'rain',
 1319328000: 'rain',
 1317427200: 'clear-day',
 1330732800: 'clear-day',
 1314403200: 'rain',
 1331942400: 'clear-day',
 1320537600: 'rain',
 1336176000: 'rain',
 1334102400: 'partly-cloudy-day',
 1324080000: 'rain',
 1328227200: 'partly-cloudy-day',
 1319846400: 'clear-day',
 1327190400: 'rain',
 1322870400: 'rain',
 1334361600: 'clear-day',
 1332633600: 'partly-cloudy-day',
 1331337600: 'partly-cloudy-day',
 1333756800: 'partly-cloudy-day',
 1321660800: 'partly-cloudy-day',
 1318550400: 'clear-day',
 1316822400: 'clear-day',
 1335571200: 'clear-day',
 1324166400: 'rain',
 1330646400: 'partly-cloudy-day',
 1331856000: 'clear-day',
 1329436800: 'rain',
 1312588800: 'rain',
 1320364800: 'clear-day',
 1316131200: 'partly-clou

In [42]:
#changing weather dictionary to a df
wdf3 = pd.DataFrame(weather_dict, index = ['weather'])

,1333152000,1323561600,1313193600,1322352000,1329523200,1327017600,1328313600,1334966400,1316304000,1319328000,...,1332288000,1333670400,1333929600,1335225600,1335744000,1335830400,1335916800,1336348800,1336435200,1336867200
weather,rain,partly-cloudy-day,partly-cloudy-day,partly-cloudy-day,partly-cloudy-day,rain,partly-cloudy-day,partly-cloudy-day,rain,rain,...,partly-cloudy-day,partly-cloudy-day,clear-day,rain,clear-day,clear-day,clear-day,partly-cloudy-day,partly-cloudy-day,partly-cloudy-day


In [43]:
#switching the axis
wdf4 = pd.DataFrame.transpose(wdf3)

In [44]:
#resetting the index and renaming the unix date column
wdf4 = wdf4.reset_index()
wdf4.rename(columns={'index': 'Unix_Date'}, inplace=True)

In [48]:
#merge weather df with original df
pd.merge(df, wdf4, how = 'left', on = 'Unix_Date')

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Unix_Date,weather
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,1333152000,rain
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,1323561600,partly-cloudy-day
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,1313193600,partly-cloudy-day
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,1322352000,partly-cloudy-day
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,1329523200,partly-cloudy-day
5,1097,D1,2011,2012-01-20,M'gladbach,Bayern Munich,3,1,H,1327017600,rain
6,1098,D1,2011,2012-02-04,Hamburg,Bayern Munich,1,1,D,1328313600,partly-cloudy-day
7,1099,D1,2011,2012-04-21,Werder Bremen,Bayern Munich,1,2,A,1334966400,partly-cloudy-day
8,1100,D1,2011,2011-09-18,Schalke 04,Bayern Munich,0,2,A,1316304000,rain
9,1101,D1,2011,2011-10-23,Hannover,Bayern Munich,2,1,H,1319328000,rain
